In [0]:
dbutils.widgets.text("customer_date","2024-07-06")
customer_date = dbutils.widgets.get("customer_date")

In [0]:
from pyspark.sql.functions import *
customer_data=f"/Volumes/workspace/zoom_car/zoom_car/zoom_car_customers_{customer_date}.json"
customer_df = spark.read.option("multiline","true").json(customer_data)
customer_df.display()

In [0]:
def isunique(customer_df,column):
    null_count = customer_df.filter(col(column).isNotNull()).count()
    if null_count > 0:
        customer_df = customer_df.filter(col(column).isNotNull())
        return customer_df
    else:
        return customer_df

df1=isunique(customer_df,"customer_id")
df2=isunique(df1,"email")
df3=isunique(df2,"name")
df3.display()

In [0]:
from pyspark.sql.functions import col, when
email_pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
clean_customer_df = df3.filter(col("email").rlike(email_pattern))
display(clean_customer_df)

In [0]:
valid_statuses = ["active", "inactive"]
clean_customer_df = clean_customer_df.filter(col("status").isin(valid_statuses))
clean_customer_df.display()

In [0]:
valid_df = clean_customer_df.filter(
    (length(regexp_replace(col("phone_number"), "\\D+", "")) == 10) &
    (~regexp_replace(col("phone_number"), "\\D+", "").startswith("0"))
)

valid_df.display()

In [0]:
df_with_date = clean_customer_df.withColumn(
    "signup_date",
    to_date(col("signup_date"), "yyyy-MM-dd")
)

# Calculate tenure in days, months, and years
tenure_df = df_with_date.withColumn(
    "tenure_days",
    datediff(current_date(), col("signup_date"))
).withColumn(
    "tenure_months",
    floor(months_between(current_date(), col("signup_date")))
).withColumn(
    "tenure_years",
    floor(months_between(current_date(), col("signup_date")) / 12)
)

tenure_df.display()

In [0]:
# tenure_df.write \
#     .format("delta") \
#     .mode("overwrite") \
#     .saveAsTable("default.zoom_car_booking")